In [1]:
from sys import path
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import math

if not ("C:\\Users\\julesl\\Documents (Local)\\JupyterLabWorkspace\\Library\\" in path) :
    path.append("C:\\Users\\julesl\\Documents (Local)\\JupyterLabWorkspace\\Library\\")

from FeedingHabitat.feedingHabitat import FeedingHabitat
%load_ext memory_profiler

In [2]:
fh = FeedingHabitat(1)

In [3]:
min, max = 0.0,10.0
fh.variables_dictionary["temperature_L1"] = np.array([[0.,0.],[0.,np.NaN]])
fh.variables_dictionary["temperature_L2"] = np.array([[0.,0.],[np.NaN,np.NaN]])
fh.variables_dictionary["temperature_L3"] = np.array([[0.,np.NaN],[np.NaN,np.NaN]])

In [4]:
print(fh.variables_dictionary["temperature_L1"])
print(fh.variables_dictionary["temperature_L2"])
print(fh.variables_dictionary["temperature_L3"])

[[ 0.  0.]
 [ 0. nan]]
[[ 0.  0.]
 [nan nan]]
[[ 0. nan]
 [nan nan]]


In [5]:
%mprun -f fh.setMask fh.setMask()

Filename: C:\Users\julesl\Documents (Local)\JupyterLabWorkspace\Library\FeedingHabitat\feedingHabitat.py

Line #    Mem usage    Increment  Occurences   Line Contents
    84    127.5 MiB    127.5 MiB           1       def setMask(self, from_Data=None) :
    85                                                 
    86    127.5 MiB      0.0 MiB           1           if from_Data is None :
    87    127.5 MiB      0.0 MiB           3               tmp_mask = (np.isfinite(self.variables_dictionary["temperature_L1"]).astype(np.int)
    88    127.5 MiB      0.0 MiB           1                                 + np.isfinite(self.variables_dictionary["temperature_L2"]).astype(np.int)
    89    127.5 MiB      0.0 MiB           1                                 + np.isfinite(self.variables_dictionary["temperature_L3"]).astype(np.int))
    90                                                     
    91    127.5 MiB      0.0 MiB           2               self.global_mask = {"mask_L1" : ((tmp_mask == 1

In [6]:
fh.global_mask

{'mask_L1': array([[ True,  True],
        [ True, False]]),
 'mask_L2': array([[ True,  True],
        [False, False]]),
 'mask_L3': array([[ True, False],
        [False, False]])}

## **Real DATA**

In [3]:
xr_ds_sst = xr.open_dataset("C:\\Users\\julesl\\Documents (Local)\\JupyterLabWorkspace\\\Data\\Concat\\SP\\MFCat_sp_2x30d_interim_historic_sst_1979_2010.nc")
xr_ds_o2 = xr.open_dataset("C:\\Users\\julesl\\Documents (Local)\\JupyterLabWorkspace\\\Data\\Concat\\SP\\MFCat_sp_2x30d_interim_historic_O2_L2_1979_2010.nc")

fh.variables_dictionary["temperature_L1"] = xr_ds_sst['sst'].data
fh.variables_dictionary["temperature_L2"] = xr_ds_sst['sst'].data
fh.variables_dictionary["temperature_L3"] = xr_ds_o2['o2'].data

%mprun -f fh.setMask fh.setMask()

Filename: C:\Users\julesl\Documents (Local)\JupyterLabWorkspace\Library\FeedingHabitat\feedingHabitat.py

Line #    Mem usage    Increment  Occurences   Line Contents
    84    146.3 MiB    146.3 MiB           1       def setMask(self, from_Data=None) :
    85                                                 
    86    146.3 MiB      0.0 MiB           1           if from_Data is None :
    87    151.0 MiB      4.7 MiB           1               tmp_mask = (np.isfinite(self.variables_dictionary["temperature_L1"]).astype(np.int) + np.isfinite(self.variables_dictionary["temperature_L2"]).astype(np.int)+ np.isfinite(self.variables_dictionary["temperature_L3"]).astype(np.int))
    88                                                     
    89    153.9 MiB      2.8 MiB           1               self.global_mask = {"mask_L1" : ((tmp_mask == 1) + (tmp_mask == 2) + (tmp_mask == 3)),"mask_L2" : ((tmp_mask == 2) + (tmp_mask == 3)),"mask_L3" : (tmp_mask == 3)}
    90                                 

## **More DATA**

In [3]:
filepath_directory = "C:\\Users\\julesl\\Documents (Local)\\JupyterLabWorkspace\\\Data\\Concat\\PO\\"

#TEMPERATURE
fh.variables_dictionary["temperature_L1"] = xr.open_dataset(filepath_directory+"MFCat_po_025x30_temperature_L1_1998_2019.nc")['temperature_L1'].data
fh.variables_dictionary["temperature_L2"] = xr.open_dataset(filepath_directory+"MFCat_po_025x30_temperature_L2_1998_2019.nc")['temperature_L2'].data
fh.variables_dictionary["temperature_L3"] = xr.open_dataset(filepath_directory+"MFCat_po_025x30_temperature_L3_1998_2019.nc")['temperature_L3'].data

In [ ]:
%mprun -f fh.setMask fh.setMask()

----
# Mask From Text File

In [4]:
fh.setMask(from_text=filepath_directory+'mask\\mask_freeglorys_025deg_corrected.txt')
fh.global_mask['mask_L1'].shape

(480, 808)

In [5]:
fh.setMask(from_text=filepath_directory+'mask\\mask_freeglorys_025deg_corrected.txt', expend_time=264)
fh.global_mask['mask_L1'].shape

(264, 480, 808)

In [8]:
not (False in  (fh.global_mask['mask_L1'][0,:,:] == fh.global_mask['mask_L1'][1,:,:]))

True